# AI Bootcamp 01 - RAG Agent by Waldemir Cambiucci

In [23]:
import os
import numpy as np
import faiss
from openai import OpenAI

In [24]:
# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY", "YOUR_API_KEY"))

In [25]:
class RAGAgent:
    def __init__(self, model="gpt-3.5-turbo", embedding_model="text-embedding-ada-002"):
        self.model = model
        self.embedding_model = embedding_model

        # Initialize FAISS index (Inner Product - IP)
        self.dimension = 1536  # Standard size for "text-embedding-ada-002"
        self.index = faiss.IndexFlatIP(self.dimension)

        # Store texts and embeddings for reference
        self.knowledge_texts = []
        self.knowledge_embeddings = None

    def embed_text(self, text):
        """Uses OpenAI's API to generate text embeddings."""
        response = client.embeddings.create(
            input=[text],
            model=self.embedding_model
        )
        return np.array(response.data[0].embedding, dtype=np.float32)

    def add_knowledge(self, texts):
        """
        Takes a list of texts (knowledge base), generates embeddings,
        and updates the FAISS index.
        """
        for text in texts:
            self.knowledge_texts.append(text)

        # Generate embeddings for each text
        embeddings = [self.embed_text(t) for t in texts]
        embeddings = np.vstack(embeddings)

        # Store embeddings
        if self.knowledge_embeddings is None:
            self.knowledge_embeddings = embeddings
        else:
            self.knowledge_embeddings = np.vstack([self.knowledge_embeddings, embeddings])

        # Add to FAISS index
        self.index.add(embeddings)

    def retrieve_relevant_chunks(self, query, top_k=3):
        """
        Takes a user query, generates an embedding, and searches FAISS
        for the most relevant text chunks.
        """
        query_embedding = self.embed_text(query).reshape(1, -1)

        # Search index (Inner Product similarity)
        scores, indices = self.index.search(query_embedding, top_k)

        # Retrieve corresponding texts
        relevant_texts = []
        for idx in indices[0]:
            relevant_texts.append(self.knowledge_texts[idx])

        return relevant_texts

    def generate_answer(self, query, relevant_texts):
        """
        Combines retrieved text chunks with the user query
        to form a prompt and calls OpenAI's API for the final answer.
        """
        context = "\n\n".join(relevant_texts)
        system_prompt = (
            "Você é um assistente especializado neste domínio. "
            "Use o contexto a seguir para responder de forma clara e objetiva.\n\n"
            f"Contexto:\n{context}\n\n"
        )
        user_prompt = f"Pergunta: {query}\nResposta:"

        completion = client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.2,
            max_tokens=300
        )

        return completion.choices[0].message.content.strip()

    def answer_query(self, query, top_k=3):
        """
        Main function:
        1. Retrieves relevant text chunks
        2. Generates a response using context
        """
        relevant_texts = self.retrieve_relevant_chunks(query, top_k=top_k)
        return self.generate_answer(query, relevant_texts)



In [28]:
# -------------------------- USAGE --------------------------
if __name__ == "__main__":
    # Example knowledge base
    knowledge_base = [
        "A linguagem Python é muito utilizada para análise de dados, inteligência artificial e desenvolvimento web.",
        "RAG (Retrieval-Augmented Generation) combina recuperação de informações relevantes e geração de texto.",
        "OpenAI oferece APIs para modelos de linguagem, como GPT-3.5, e para embeddings, como text-embedding-ada-002.",
        "FAISS é uma biblioteca de código aberto para indexação e busca vetorial eficiente."
    ]

    # Instantiate and inject knowledge
    agent = RAGAgent()
    agent.add_knowledge(knowledge_base)

    # Example query
    # user_query = "O que é RAG e como posso usar Python para implementar?"
    user_query = "Explique a biblioteca FAISS."

    # Generate response
    answer = agent.answer_query(user_query, top_k=2)
    print("Pergunta:", user_query)
    print("Resposta:", answer)


Pergunta: Explique a biblioteca FAISS.
Resposta: A biblioteca FAISS é uma ferramenta de código aberto utilizada para indexação e busca eficiente de vetores. Ela é amplamente utilizada em aplicações que envolvem grandes conjuntos de dados vetoriais, como processamento de linguagem natural, reconhecimento de imagens e recomendação de conteúdo. A FAISS oferece algoritmos otimizados para realizar operações de busca e recuperação de informações de forma rápida e eficaz.
